In [1]:
# %load import_setup.py
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

import os
os.chdir("../")

%load_ext autoreload
%autoreload 2

RANDOM_STATE = 42  # for reproducibility
np.random.seed(RANDOM_STATE)

NUM_ELECTRODES = 64
FS = 240          # Hz
NUM_TRAIN_LETTERS = 85
NUM_TEST_LETTERS = 100
NUM_ROWCOLS = 12    # col:1-6 row:7-12
NUM_REPEAT = 15
SECONDS_TO_SLICE = 0.65    # after the simulation 0.65s data is treated as a sample
DATA_DIR = "./data/raw/BCI_Comp_III_Wads_2004/"

In [2]:
# %load data_preparation.py
from src import pipelines

A_train = pipelines.signal_mat_sub_band_norm(DATA_DIR+"Subject_A_Train")
A_test = pipelines.signal_mat_sub_band_norm(DATA_DIR + "Subject_A_Test")

A_train['signal'] = A_train['signal'].reshape([-1,NUM_ELECTRODES,A_train['signal'].shape[-1],1])
A_test['signal'] = A_test['signal'].reshape([-1,NUM_ELECTRODES,A_test['signal'].shape[-1],1]) 

from keras import layers, models
from keras.utils import to_categorical

A_train['label'] = to_categorical(A_train['label']) 

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(A_train['signal'], A_train['label'], test_size=0.05,
                                                  random_state=RANDOM_STATE, stratify=A_train['label'])



Using TensorFlow backend.


In [3]:
from src.models.PAMI import CNN_1_P300_PAMI_BCIIII
model_PAMI = CNN_1_P300_PAMI_BCIIII(Ns = 10,seconds_to_slice=SECONDS_TO_SLICE)
model_PAMI.compile(optimizer = 'sgd',loss = 'binary_crossentropy',metrics = ['acc', 'mse'])

In [4]:
from keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor = "val_mean_squared_error",patience = 10)

In [5]:
model_PAMI.fit(x = X_train, y = y_train, batch_size=32, epochs = 1000, callbacks = [earlystopping], 
              validation_data = [X_val,y_val])

Train on 14535 samples, validate on 765 samples
Epoch 1/1000
14535/14535 [==============================] - 8s - loss: 0.4605 - acc: 0.8273 - mean_squared_error: 0.1433 - val_loss: 0.4316 - val_acc: 0.8405 - val_mean_squared_error: 0.1320
Epoch 2/1000
14535/14535 [==============================] - 4s - loss: 0.4489 - acc: 0.8290 - mean_squared_error: 0.1393 - val_loss: 0.4244 - val_acc: 0.8405 - val_mean_squared_error: 0.1298
Epoch 3/1000
14535/14535 [==============================] - 4s - loss: 0.4440 - acc: 0.8290 - mean_squared_error: 0.1378 - val_loss: 0.4170 - val_acc: 0.8405 - val_mean_squared_error: 0.1279
Epoch 4/1000
14535/14535 [==============================] - 4s - loss: 0.4396 - acc: 0.8290 - mean_squared_error: 0.1366 - val_loss: 0.4113 - val_acc: 0.8405 - val_mean_squared_error: 0.1261
Epoch 5/1000
14535/14535 [==============================] - 4s - loss: 0.4350 - acc: 0.8290 - mean_squared_error: 0.1352 - val_loss: 0.4053 - val_acc: 0.8405 - val_mean_squared_error: 0.12

In [7]:
from src.pipelines import test_pipeline,PARADIGM,accuracy
predictions = test_pipeline(A_test['signal'],A_test['code'],model_PAMI,15,PARADIGM)

from src.pipelines import accuracy
accuracy(predictions,'WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU')

0.92

In [8]:
model_PAMI_2 = CNN_1_P300_PAMI_BCIIII(Ns = 10,seconds_to_slice=SECONDS_TO_SLICE)
model_PAMI_2.compile(optimizer = 'sgd',loss = 'binary_crossentropy',metrics = ['acc', 'mse'])

earlystopping_2 = EarlyStopping(monitor = "val_mean_squared_error",patience = 3)

model_PAMI_2.fit(x = X_train, y = y_train, batch_size=32, epochs = 1000, callbacks = [earlystopping_2], 
              validation_data = [X_val,y_val])

predictions = test_pipeline(A_test['signal'],A_test['code'],model_PAMI_2,15,PARADIGM)
accuracy(predictions,'WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU')

Train on 14535 samples, validate on 765 samples
Epoch 1/1000
14535/14535 [==============================] - 6s - loss: 0.4629 - acc: 0.8258 - mean_squared_error: 0.1443 - val_loss: 0.4307 - val_acc: 0.8405 - val_mean_squared_error: 0.1318
Epoch 2/1000
14535/14535 [==============================] - 4s - loss: 0.4482 - acc: 0.8290 - mean_squared_error: 0.1391 - val_loss: 0.4233 - val_acc: 0.8405 - val_mean_squared_error: 0.1294
Epoch 3/1000
14535/14535 [==============================] - 4s - loss: 0.4418 - acc: 0.8290 - mean_squared_error: 0.1372 - val_loss: 0.4151 - val_acc: 0.8405 - val_mean_squared_error: 0.1269
Epoch 4/1000
14535/14535 [==============================] - 5s - loss: 0.4349 - acc: 0.8290 - mean_squared_error: 0.1351 - val_loss: 0.4072 - val_acc: 0.8405 - val_mean_squared_error: 0.1246
Epoch 5/1000
14535/14535 [==============================] - 4s - loss: 0.4269 - acc: 0.8290 - mean_squared_error: 0.1327 - val_loss: 0.3980 - val_acc: 0.8405 - val_mean_squared_error: 0.12

0.93

In [12]:
model_PAMI_2.history.history

{'acc': [0.82583419335925956,
  0.82903336774536041,
  0.82903336773305814,
  0.82903336774125969,
  0.82903336774125969,
  0.82906776749066646,
  0.83020295837633296,
  0.83130374957820496,
  0.83278293776930423,
  0.83477812180783195,
  0.83543171657907378,
  0.83708290333677327,
  0.83814929483844769,
  0.83952528383053571,
  0.84031647748458327,
  0.84103887168902647,
  0.84375644998120658,
  0.84413484692122465,
  0.84434124527003784,
  0.8458548331613347,
  0.84554523565861872,
  0.84764361888385531,
  0.84733402136063551,
  0.84919160646714831,
  0.84884760921912628,
  0.85018919848641206,
  0.85053319575493791,
  0.85005159959540488],
 'loss': [0.46286677810839866,
  0.4482021279174272,
  0.44180694344448068,
  0.43489483228695935,
  0.426855905699,
  0.41820377556983246,
  0.41035331958064847,
  0.40366161060284045,
  0.39829230537765575,
  0.39400341278464268,
  0.39045997343401256,
  0.38742242677333966,
  0.38433829045271112,
  0.38173253696154269,
  0.3786403203363225,
  0